#### Итак, парсинг сайта avito, или же подготовка к проекту по Информационному поиску 2018.

Сперва импортиуем необходимые модули.

Запросы осуществляются с помощью **requests**

Парсинг HTML кода с помощью **BeautifulSoup**

**time.sleep** для того, чтобы нас не банил сайт

**pickle** для промежуточного сохранения результатов

**datetime** для форматирования даты в объявлении

In [ ]:
import requests
from bs4 import BeautifulSoup
from time import sleep
import random
import pickle
import warnings
import datetime

warnings.filterwarnings('ignore')

Сперва я решил выкачить 10000 ссылок. Чтобы если что-то сломается, то не все будет потеряно.

В функции:
    
    data – Список, куда складывается ссылки
    root – Основание ссылок
    pages – Количество страниц, которые надо обойти программе

In [ ]:
def get_links(data, root, pages):
    print('...Function init...')
    for page in range(1, pages):
        local_link = '{}/rossiya/mototsikly_i_mototehnika?p={}&view=list'.format(root, page)
        print('...Link {} in process...'.format(local_link))
        soup = BeautifulSoup(requests.get(local_link).text, 'html')
        for item in soup.find_all('a', attrs={'class':'description-title-link'}):
            data.append('{}{}'.format(root, item.get('href')))
        print('...Link parsed...'.format(local_link))
        sleep(random.randint(8, 10))

In [ ]:
get_links(links, main_link, 120)

Теперь проверим скольку у нас уникальных ссылок и все ли они содержат нашу тематику.

In [ ]:
links = list(set(links))

print(len(links))

for link in links:
    if 'mototsikly_i_mototehnika' in link:
        continue
    else:
        print('False')

Сохраним ссылки, чтобы не потерять наш прогресс

In [ ]:
with open('links.pickle', 'wb') as f:
    pickle.dump(links, f)

Теперь нам неоходимо достать информацию. Создади словарь месяцев для обработки даты и функции для вытяивания информации с помощью BS4

In [ ]:
month = {'января':'01',
         'февраля':'02',
         'марта':'03',
         'апреля':'04',
         'мая':'05',
         'июня':'06',
         'Июля':'07',
         'августа':'08',
         'сентября':'09',
         'октября':'10',
         'ноября':'11',
         'декабря':'12'}


def get_date(soup):
    try:
        line = (soup.find('div',
                          attrs={'class':'title-info-metadata-item'}).get_text())
        date = line.split('размещено')[1]
        if 'сегодня' in date:
            return datetime.date.today()
        elif 'вчера' in date:
            return datetime.date.today() - datetime.timedelta(1)
        else:
            date = date.split()
            return '2018-{}-{}'.format(month[date[1]], date[0])
    except AttributeError:
        return 'None'

    
def get_title(soup):
    try:
        line = soup.find('span',
                         attrs={'class':'title-info-title-text'}).get_text()
        return line
    except AttributeError:
        return 'None'
    
    
def get_title(soup):
    try:
        line = soup.find('span',
                         attrs={'class':'title-info-title-text'}).get_text()
        return line
    except AttributeError:
        return 'None'
    

def get_address(soup):
    try:
        address = soup.find('span',
                            attrs={'itemprop':'streetAddress'}).get_text().replace('\n', '')
        return address
    except AttributeError:
        address = 'None'

        
def get_price(soup):
    try:
        price = '{}{}'.format(soup.find('span',
                                        attrs={'class':'js-item-price'}).get_text(),
                              soup.find('span',
                                        attrs={'class':'font_arial-rub'}).get_text())
        return price
    except AttributeError:
        return 'None'
    
    
def get_description(soup):
    try:
        description = soup.find('div',
                                attrs={'class':'item-description-text'}).get_text().replace('\n', '')
        return description
    except AttributeError:
        return 'None'

Теперь напишим функцию для формирования самого текста, который я буду записывать в файлы

In [ ]:
def form_text(link):
    soup = BeautifulSoup(requests.get(link).text, 'html')
    date = get_date(soup)
    address = get_address(soup)
    title = get_title(soup)
    price = get_price(soup)
    description = get_description(soup)
    result = 'Title:\t{}\nDate:\t{}\nAdress:\t{}\nPrice:\t{}\nComment:{}\n'.format(title,
                                                                                     date,
                                                                                     address,
                                                                                     price,
                                                                                     description)
    return result

Проверка работоспособности...

In [ ]:
print(form_text('https://www.avito.ru/kaliningrad/mototsikly_i_mototehnika/daelim_roadwin_125_2008_g._v_465758898'))

Запускаем цикл загрузки и ждем окончания работы нашей программы.

In [ ]:
for link in links:
    file_name = link.split('/')[len(link.split('/')) - 1]
    with open('avito_files/' + str(file_name) + '.txt', 'w') as f:
        try:
            f.write(form_text(link))
            f.write('Link:\t{}\n'.format(str(link)))
            print('Link {} parsed. Saved as "{}"'.format(num, str(file_name)))
        except IndexError:
            print('>>> BROKEN LINK N.{} <<<'.format(num))
        num += 1
        sleep(random.randint(8, 10))

С реальной работой данной программы можно ознакомиться в файле avito.ipynb.
Возможно, там код немного отличается, но делает он абсолютно то же)